In [8]:
# https://milvus.io/docs/hybrid_search_with_milvus.md
# BGE-M3: https://huggingface.co/BAAI/bge-m3

# https://milvus.io/docs/chinese-analyzer.md 中文用的jieba
# jieba库好多年没有更新过了 而且对英文的支持也不好

下面是 本 session 对话内容的结构化总结笔记（中文、清晰、有组织，可直接放进你的项目文档里）。

⸻

📝 Milvus + BGE-M3 + 中文/英文全文检索体系：对话总结笔记

1. 用户问题起点：

“Milvus 如何做中文全文检索？有没有多语言 analyzer？jieba 可处理中英混合吗？是否还需要 Elasticsearch？”

⸻

2. Milvus 的全文检索能力本质

Milvus 的 “full-text search” 能力 不是传统搜索引擎级别：
	•	❌ 没有 tokenizer pipeline
	•	❌ 没有 BM25
	•	❌ 没有多语言 analyzer
	•	❌ 不支持 ik / jieba / smartcn / Lucene analyzers
	•	✔ 只支持简单的 substring 匹配
	•	✔ 支持向量搜索（dense search）
	•	✔ 现在新增 稀疏向量（sparse vector）索引，可模拟 lexical 检索

👉 因此：Milvus 的全文检索 本质是向量检索 + 稀疏倒排索引（由模型生成），不是依赖分词器。

⸻

3. jieba 的现状与能力
	•	✔ jieba 能做中文分词
	•	✔ 英文会作为一个 token 处理
	•	✔ 中英文混合能处理但很粗糙
	•	❌ 不做英文的 NLP tokenization（词形还原、复数等）
	•	❌ 多年不更新（处于停滞状态）
	•	❌ 不适合严肃全文检索
	•	❌ 不适合现代 RAG / hybrid retrieval 系统

结论：jieba 可以作为小工具，不能作为核心全文检索方案。

⸻

4. 什么是 BGE-M3？（对话中的重点）

BGE-M3 是 北京智源发布的全球最强的通用 embedding 模型之一，特点：

M3 = Multi-Function + Multi-Lingual + Multi-Granularity

它同时输出：
	•	dense embedding（语义向量） → Milvus dense search
	•	sparse embedding（稀疏向量） → Milvus sparse inverted index（相当于 BM25-like）

优势：
	•	✔ 原生支持中英文混合
	•	✔ 支持多语言
	•	✔ 支持 lexical + semantic
	•	✔ 用 sparse embedding 替代传统分词器（不再需要 jieba / ik）
	•	✔ 非常适合 RAG、教育内容、智库文档、多语言技术文本

总结：BGE-M3 = 分词器 + BM25 替代 + 语义模型 + 多语言模型。

⸻

5. 关键突破点：Milvus 官方已经原生支持 BGE-M3（dense + sparse + hybrid）

你发现的 Milvus 文档说明：

Milvus 现在直接支持 dense search + sparse search + hybrid search，且官方提供了 BGEM3EmbeddingFunction。

流程：
	1.	文本 → BGE-M3
	•	dense vector
	•	sparse vector
	2.	Milvus 建 collection：
	•	FLOAT_VECTOR
	•	SPARSE_FLOAT_VECTOR（新能力）
	3.	查询时：
	•	dense 搜索（语义）
	•	sparse 搜索（关键词）
	•	hybrid 搜索（得分加权融合）

效果 = BM25 + 向量语义 的组合搜索，但无需 Elasticsearch。

👉 Milvus 现在自身就能做 hybrid search，因此不一定非要 ES 了。

⸻

6. 是否还需要 Elasticsearch？

✔ 可以不需要 ES 的场景（你当前就是）
	•	RAG 检索
	•	文本 QA
	•	教育/论文/智库资料检索
	•	中英文混合文本
	•	重点是召回与语义，不是复杂 DSL

Milvus + BGE-M3 足够优秀，架构简单，还更现代。

⸻

❗ 仍然需要 ES 的情况

如果你未来要：
	•	复杂过滤：must / must_not / filter
	•	多层嵌套 bool 查询
	•	聚合统计（按年份、机构、课题分类等）
	•	正则/通配符/pinyin 搜索
	•	非文本字段排序（时间权重 + 热度）
	•	大规模 BI 分析

→ ES 仍然不可替代。

现在阶段（你的智库系统 + RAG + 教材知识库）：

👉 完全可以不加 ES。

⸻

7. 最终系统结构推荐（结合你的场景）

⭐ 推荐架构（最适合你当前项目）

PDF / Word / 教材内容
      ↓
   文本解析
      ↓
    chunk
      ↓
 BGE-M3 (dense + sparse)
      ↓
 ┌────────────┐
 │   Milvus   │
 │ dense index│  ← semantic retrieval
 │ sparse idx │  ← keyword-like retrieval
 │ hybrid     │  ← dense + sparse (强大!)
 └────────────┘
      ↓
   RAG / QA

这是目前最现代、最实用、最适合中文/英文技术文本的系统架构。

⸻

🔚 本 session 的一句话总结

Milvus + BGE-M3 已经可以完成传统“全文检索 + 语义检索”的混合任务，在当前你的 RAG / 智库内容检索需求中，可以完全不使用 Elasticsearch。jieba 已经过时，BGE-M3 的 sparse embedding 已经自然替代了分词器。

⸻

如果你愿意，我还能帮你继续生成：
	•	你的 Milvus + BGE-M3 collection schema 最佳模板
	•	端到端 ingest pipeline（PDF→chunk→embedding→Milvus）
	•	hybrid_search(weight_dense, weight_sparse) 查询接口设计
	•	小型 demo 代码

叫我继续即可。

In [9]:
##  Hybrid Search with Milvus
# BGE-M3 model can convert text into dense and sparse vectors.
# - Dense Retrieval: Utilizes semantic context to understand the meaning behind queries.
# - Sparse Retrieval: Emphasizes keyword matching to find results based on specific terms, equivalent to full-text search.
# - Hybrid Retrieval: Combines both Dense and Sparse approaches, capturing the full context and specific keywords for comprehensive search results.

In [10]:
## Load and Prepare Data
import pandas as pd

file_path = "quora_duplicate_questions.tsv"
df = pd.read_csv(file_path, sep="\t")
questions = set()
for _, row in df.iterrows():
    obj = row.to_dict()
    questions.add(obj["question1"][:512])
    questions.add(obj["question2"][:512])
    if len(questions) > 500:  # Skip this if you want to use the full dataset
        break

docs = list(questions)

# example question
print(docs[0])

When a girlfriend asks her boyfriend "Why did you choose me? What makes you want to be with me?", what should one reply to her?


In [11]:
## Use BGE-M3 Model for Embeddings
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = ef.dim["dense"]

# Generate embeddings using BGE-M3 model
docs_embeddings = ef(docs)

pre tokenize: 100%|██████████| 32/32 [00:00<00:00, 1183.79it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 32/32 [00:05<00:00,  5.57it/s]


In [12]:
## Setup Milvus Collection and Index
# 


from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

# milvus lite
connections.connect(uri="./milvus.db")

fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
]
schema = CollectionSchema(fields)

col_name = "hybrid_demo"
if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Bounded")

sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()


In [13]:
## Insert Data into Milvus Collection

# For efficiency, we insert 50 records in each small batch
for i in range(0, len(docs), 50):
    batched_entities = [
        docs[i : i + 50],
        docs_embeddings["sparse"][i : i + 50],
        docs_embeddings["dense"][i : i + 50],
    ]
    col.insert(batched_entities)
print("Number of entities inserted:", col.num_entities)


Number of entities inserted: 502


In [14]:
## Enter Your Search Query

# Enter your search query
query = input("Enter your search query: ")
print(query)

# Generate embeddings for the query
query_embeddings = ef([query])
# print(query_embeddings)

How to start learning programming?


In [15]:
# Run the Search
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)


def dense_search(col, query_dense_embedding, limit=10):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]


def sparse_search(col, query_sparse_embedding, limit=10):
    search_params = {
        "metric_type": "IP",
        "params": {},
    }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]


def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=10,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
    )[0]
    return [hit.get("text") for hit in res]


In [16]:
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"][[0]])
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"][[0]],
    sparse_weight=0.7,
    dense_weight=1.0,
)

In [18]:
# Display Search Result
def doc_text_formatting(ef, query, docs):
    tokenizer = ef.model.tokenizer
    query_tokens_ids = tokenizer.encode(query, return_offsets_mapping=True)
    query_tokens = tokenizer.convert_ids_to_tokens(query_tokens_ids)
    formatted_texts = []

    for doc in docs:
        ldx = 0
        landmarks = []
        encoding = tokenizer.encode_plus(doc, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])[1:-1]
        offsets = encoding["offset_mapping"][1:-1]
        for token, (start, end) in zip(tokens, offsets):
            if token in query_tokens:
                if len(landmarks) != 0 and start == landmarks[-1]:
                    landmarks[-1] = end
                else:
                    landmarks.append(start)
                    landmarks.append(end)
        close = False
        formatted_text = ""
        for i, c in enumerate(doc):
            if ldx == len(landmarks):
                pass
            elif i == landmarks[ldx]:
                if close:
                    formatted_text += "</span>"
                else:
                    formatted_text += "<span style='color:red'>"
                close = not close
                ldx = ldx + 1
            formatted_text += c
        if close is True:
            formatted_text += "</span>"
        formatted_texts.append(formatted_text)
    return formatted_texts


In [19]:
# Then we can display search results in text with highlights:
from IPython.display import Markdown, display

display(Markdown("**Dense Search Results:**"))
formatted_results = doc_text_formatting(ef, query, dense_results)
for result in dense_results:
    display(Markdown(result))

display(Markdown("\n**Sparse Search Results:**"))
formatted_results = doc_text_formatting(ef, query, sparse_results)
for result in formatted_results:
    display(Markdown(result))

display(Markdown("\n**Hybrid Search Results:**"))
formatted_results = doc_text_formatting(ef, query, hybrid_results)
for result in formatted_results:
    display(Markdown(result))


**Dense Search Results:**

What's the best way to start learning robotics?

How do I learn a computer language like java?

How can I get started to learn information security?

What is Java programming? How To Learn Java Programming Language ?

...

<span style='color:red'>How</span> can I learn computer security<span style='color:red'>?</span>

<span style='color:red'>How</span> can I learn<span style='color:red'> to</span> speak English fluently<span style='color:red'>?</span>

What are the best ways<span style='color:red'> to</span> learn French<span style='color:red'>?</span>

<span style='color:red'>How</span> can you make physics easy<span style='color:red'> to</span> learn<span style='color:red'>?</span>

<span style='color:red'>How</span> do we prepare for UPSC<span style='color:red'>?</span>